In [143]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

tf.compat.v1.disable_eager_execution()

In [144]:
train, test = tf.keras.datasets.mnist.load_data()

train_x, train_y = train
train_x = np.expand_dims(train_x.astype(np.float32), axis = 3)
print('X shape : ', train_x.shape)

X shape :  (60000, 28, 28, 1)


In [145]:
num_input = 784
num_classes = 10

X = tf.compat.v1.placeholder(tf.float32, [None, 28,28,1])
y = tf.compat.v1.placeholder(tf.float32, [None, num_classes])

In [146]:
def conv2d(x, w, b, strides = 1):
    x = tf.nn.conv2d(x, w, strides = [1,strides,strides,1], padding = 'SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2, s=1):
    x = tf.nn.max_pool(x, ksize = [1,k,k,1], strides = [1,s,s,1], padding = 'SAME')
    return x

def net(x, weights, biases):
    x = conv2d(x, weights[0], biases[0])
    x = maxpool2d(x)

    x = conv2d(x, weights[1], biases[1])
    x = maxpool2d(x)
    print(x.shape)

    x = tf.compat.v1.layers.flatten(x)

    x = tf.add(tf.matmul(x, weights[2]), biases[2])
    x = tf.nn.relu(x)

    out = tf.add(tf.matmul(x, weights[3]), biases[3])

    return out

In [147]:
fc_input_shape = 28

weights = [
    # [filter width, filter height, in_channels, out_channels]
    tf.Variable(tf.random.normal([5,5,1,32])),
    tf.Variable(tf.random.normal([5,5,32,64])),

    # input shape (height,width,channels), out_neurons
    tf.Variable(tf.random.normal([fc_input_shape * fc_input_shape * 64, 1024])),
    tf.Variable(tf.random.normal([1024,num_classes]))
]

biases = [
    tf.Variable(tf.random.normal([32])),
    tf.Variable(tf.random.normal([64])),
    tf.Variable(tf.random.normal([1024])),
    tf.Variable(tf.random.normal([num_classes]))
]
temp_arr = np.array([weights,biases])
temp_arr = temp_arr.reshape((temp_arr.shape[0] * temp_arr.shape[1]))
temp_arr = list(temp_arr)


In [148]:
conv_net = net(X, weights, biases)
preds = tf.nn.softmax(conv_net)
opt = tf.optimizers.Adam(0.001)

#loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = conv_net, labels = y))
loss_op = tf.keras.losses.CategoricalCrossentropy()

with tf.GradientTape() as tape : 
    loss = loss_op(preds,y)
    params = temp_arr
    grads = tape.gradient(loss,params)
    opt.apply_gradients(zip(grads,params))

#step = opt.minimize(loss, [weights,biases])

(None, 28, 28, 64)


ValueError: No gradients provided for any variable: ['Variable_170:0', 'Variable_171:0', 'Variable_172:0', 'Variable_173:0', 'Variable_174:0', 'Variable_175:0', 'Variable_176:0', 'Variable_177:0'].